In [1]:
import os
import cProfile

from cities.queries.causal_insight import CausalInsight
from cities.utils.data_grabber import DataGrabber, list_interventions, list_outcomes

smoke_test = "CI" in os.environ
num_samples = 10 if smoke_test else 1000

In [2]:
interventions = list_interventions()
outcomes = list_outcomes()
shifts = [1, 2, 3]

outcome = "unemployment_rate"
outcome2 = "gdp"
intervention = "spending_commerce"
intervention2 = "spending_transportation"
intervened_value = 0.7

data = DataGrabber()
data.get_features_wide(["gdp"])
gdp = data.wide["gdp"]
fips = gdp["GeoFIPS"][5]  # 1011
fips2 = gdp["GeoFIPS"][6]  # 1013

In [3]:
def basic_run():
    ci = CausalInsight(
        outcome_dataset=outcome,
        intervention_dataset=intervention,
        num_samples=num_samples,
        smoke_test=smoke_test,
    )

    ci.generate_tensed_samples()
    ci.get_fips_predictions(intervened_value=intervened_value, fips=fips)
    ci.plot_predictions(range_multiplier=1)

In [4]:
profiler_basic = cProfile.Profile()

profiler_basic.enable()
basic_run()
profiler_basic.disable()

# confirming that the time consuming moves involve the model
# gettign samples, etc.
profiler_basic.print_stats(sort="cumulative")

# note: restricting sites to tau drops time from 8s to 5s.

smoke test: True True


         1151773 function calls (1121247 primitive calls) in 3.617 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.619    1.809 interactiveshell.py:3397(run_code)
    792/2    0.003    0.000    3.618    1.809 {built-in method builtins.exec}
        1    0.001    0.001    3.618    3.618 433866942.py:1(<module>)
        1    0.001    0.001    3.617    3.617 4044749442.py:1(basic_run)
        1    0.000    0.000    2.619    2.619 causal_insight.py:107(generate_tensed_samples)
        3    0.002    0.001    2.531    0.844 causal_insight.py:65(generate_samples)
  323/167    0.002    0.000    2.323    0.014 _decorators.py:323(wrapper)
      132    0.001    0.000    2.293    0.017 _decorators.py:170(wrapper)
      132    0.005    0.000    2.291    0.017 readers.py:854(read_csv)
      132    0.014    0.000    2.283    0.017 readers.py:571(_read)
        3    0.018    0.006    2.216    0.739 modeli

In [5]:
# this is prep pipeline
print(smoke_test)
ci = CausalInsight(
    outcome_dataset=outcome,
    intervention_dataset=intervention2,
    num_samples=num_samples,
    smoke_test=smoke_test,
)
ci.generate_tensed_samples()

True
True True


In [6]:
# this is the slim execution
def slim_run():
    ci = CausalInsight(
        outcome_dataset=outcome,
        intervention_dataset=intervention,
        num_samples=num_samples,
    )

    ci.get_tau_samples()
    ci.get_fips_predictions(intervened_value=intervened_value, fips=fips)
    ci.plot_predictions(range_multiplier=1)


profiler_slim = cProfile.Profile()

profiler_slim.enable()
slim_run()
profiler_slim.disable()


profiler_slim.print_stats(sort="cumulative")

None None


         330674 function calls (316654 primitive calls) in 0.667 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.667    0.333 interactiveshell.py:3397(run_code)
        2    0.000    0.000    0.667    0.333 {built-in method builtins.exec}
        1    0.001    0.001    0.667    0.667 125765549.py:2(slim_run)
   121/63    0.000    0.000    0.580    0.009 _decorators.py:323(wrapper)
       50    0.000    0.000    0.573    0.011 _decorators.py:170(wrapper)
       50    0.001    0.000    0.573    0.011 readers.py:854(read_csv)
       50    0.002    0.000    0.571    0.011 readers.py:571(_read)
        1    0.001    0.001    0.562    0.562 causal_insight.py:131(get_fips_predictions)
        1    0.003    0.003    0.455    0.455 modeling_utils.py:20(prep_wide_data_for_inference)
       50    0.001    0.000    0.419    0.008 readers.py:1762(read)
        7    0.000    0.000    0.416    0.059 data